In [77]:
__author__ = 'Vamshi Guduguntla'
__copyright__ = "NA"
__license__ = "NA"
__version__ = "1.0"

from __future__ import print_function, division
from time import strftime
from pprint import pprint
import math,random,numpy as np



class Kursawe:
    
    """
    :param num_decisions    - Number of decisions/dimension of the array
    :param num_objectives   - Number of objectives to evaluate (f1,f2,....)
    :param dec_high         - Max range of the decision
    :param dec_low          - Min range of the decision
    :param obj_high         - Max range of the objective
    :param obj_low          - Min range of the objective
    :param num_objectives   - Number of objectives to evaluate (f1,f2,....)
    :param evals            - Keeps the track of the number of evaluations
    
    :func  function_value   - A function that scores a candidate
    :func  constraint_ok    - A function that checks for constraints (none of Kursawe and Schaffer)
    :func  norm             - A function that computes the objective scores for each candidate(normalized)
    :func  reset_baseline   - Resets the baseline for obj_high and obj_low computation
    """
    
    def __init__(self,num_eval = 0.0,prob = 0.5,high = 10**6,low = -10**6):
        self.name = "Kurasawe"
        self.num_decisions = 3
        self.num_objectives = 2
        self.p = prob
        self.dec_high = [5 for _ in range(self.num_decisions)]
        self.dec_low = [-5 for _ in range(self.num_decisions)]
        self.steps = 10
        self.evals = num_eval
        self.obj_high = high
        self.obj_low = low
        self.threshold = -100
        self.current_state = self.randomstate() 
        self.reset_baseline()
        self.threshold = self.obj_low
        
        
    def reset_baseline(self):
        high = -10**6
        low = 10**6
        for _ in range(1000):
            dec = self.randomstate()
            en = self.function_value(dec)
            
            if en > high:
                high = en
            if en < low:
                low = en
        
        self.obj_high = high
        self.obj_low = low
        
    def contraint_ok(self,dec):
        return True
        
    def randomstate(self):
        while True:
            dec = list()
            for low,high in zip(self.dec_low,self.dec_high):
                dec.append(random.randrange(low,high))
            if self.contraint_ok(dec):
                return dec   
        
    def function_value(self,dec):
        f1,f2 = 0.,0.
        for i in range(self.num_decisions - 1):
            f1 += (-10) * math.exp((-0.2) * math.sqrt(dec[i]**2 + dec[i+1] ** 2))
        for i in range(self.num_decisions):
            f2 += math.fabs(dec[i]) + 5 * math.sin(dec[i])
        return f1+f2 
    
    def norm(self,state):
        return (self.function_value(state) - self.obj_low)/(self.obj_high - self.obj_low)

In [84]:
class MWS:
    
    """
    :param max_tries                     - maximum retries
    :param max_changes                   - maximum iterations
       
    :func  mutate_in_a_dimension         - A function mutates the candidate solution in one dimension
    :func  mutate_to_maximize            - A function mutates to maximize the objective by incremental steps
    :func  maxwalksat                    - Initiates the maxwalksat algorithm
    """
    
    
    def __init__(self,model):
        self.notation= """Notation:
        "?" - moved to a worse solution/ random jump
        "." - solution that does not change
        "+" - better solution using a local change along one dimension"""
        self.max_tries = 100
        self.max_changes = 50
        self.maxwalksat(model)
    
    def mutate_in_a_dimension(self,state):
        rand_index = random.randrange(0,model.num_decisions - 1)
        temp = list(state)
        if model.p < random.random():
            temp[rand_index] = random.randrange(model.dec_low[rand_index],model.dec_high[rand_index])
            if model.contraint_ok(temp):
                return temp, "."
            else:
                return state,"?"
        else:
            temp = self.mutate_to_maximize(temp,rand_index)
            if temp == state:
                return temp,"."
            else:
                return temp,"+"
    
    
    def mutate_to_maximize(self,state,rand_index):
        increment = (model.dec_high[rand_index] - model.dec_low[rand_index])/model.steps
        temp = list(state)
        best = state
        for _ in range(model.steps):
            temp[rand_index] += increment
            model.evals += 1
            if (model.function_value(temp) < model.function_value(best)) and model.contraint_ok(temp):
                best = list(temp)
        state = best
        return state
    
    
    def maxwalksat(self,model):
        model.reset_baseline()
        print("Minimizing ",model.name)
        print("Objective High :",model.obj_high)
        print("Objective Low  :",model.obj_low)
        print("\n")
        print(self.notation)
        print("\n")
        best_state = model.randomstate()
        for _ in range(self.max_tries):
            state = model.randomstate()
            display = str()
            for i in range(self.max_changes):
                if model.function_value(state) < model.threshold:
                    return state
                else:
                    new_state, symbol = self.mutate_in_a_dimension(state)
                display += symbol
                if(model.function_value(state)<model.function_value(best_state)):
                    best_state = state
                if(model.function_value(new_state)<model.function_value(best_state)):
                    best_state = new_state
            print(display,round(model.norm(best_state),5))
        return np.around(best_state,9),"f1+f2:",model.function_value(best_state),"Evals: ",model.evals


In [85]:
model = Kursawe()
MWS(model)

Minimizing  Kurasawe
Objective High : 22.1634516407
Objective Low  : -24.694831101


Notation:
        "?" - moved to a worse solution/ random jump
        "." - solution that does not change
        "+" - better solution using a local change along one dimension


.+...+.+++............++.....+..+..+..+........... 0.19256
.................................................. 0.19256
++.+.+....+..++++..+...+.++...+.+++.....++..+.+.+. 0.19256
.++++++++++.+.+++++++.+....+.++.++++..+...+.++.+.+ 0.19256
...........++.++..........+....+.++.....+.....+... 0.17881
+.++..++++..++..+++.++..+.+++++.++..+.+++......+.+ 0.17881
+.+..+..+++..++.+..+..+++.+.+.+++.++..+.++++..+... 0.17881
++......................++.+..+.+++..+..+.+...++.+ 0.17881
++++....+...++..+.++.+++.+.+...+++.+..+..+.+.+++.. 0.17881
++++++++...+..+..+.+..+.+++.+.+...+.++.++...+++.++ 0.17881
..++....+...++.+.+..+.+.+.++...++..+.++...+.+++... 0.17881
......++++........++........+..+...++....++.+.+... 0.03848
......++.++.....+.++.+.+...

<__main__.MWS instance at 0x7fa93c0712d8>